In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/ajwu/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
exoplanet_df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
exoplanet_df = exoplanet_df.dropna(axis='columns', how='all')
# Drop the null rows
exoplanet_df = exoplanet_df.dropna()
exoplanet_df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
exoplanet_df2 = exoplanet_df.drop("koi_disposition", axis = 1)
selected_features = exoplanet_df2.columns
selected_features

Index(['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk',
       'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1',
       'koi_impact_err2', 'koi_duration', 'koi_duration_err1',
       'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2',
       'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_insol',
       'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg',
       'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad', 'koi_srad_err1',
       'koi_srad_err2', 'ra', 'dec', 'koi_kepmag'],
      dtype='object')

# Model Selection: Support Vector Machine

In [6]:
# Assign exoplanet dataframe to X and y.
# Filtered X values down, eliminating features where the feature score was less than 0.02. 
# Use `koi_disposition` for the y values

X = exoplanet_df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_model_snr', 'koi_steff_err1',
       'koi_steff_err2','koi_kepmag']]

y = exoplanet_df["koi_disposition"]



# Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify = y)

In [8]:
X_train.head()

koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  koi_period  \
4002              0              0              1              0   99.673478   
4246              0              1              0              0    0.592244   
548               0              1              1              0    9.991625   
3953              0              1              0              0  178.412990   
2362              0              0              0              0   45.294223   

      koi_time0bk_err1  koi_time0bk_err2  koi_duration  koi_duration_err1  \
4002          0.002300         -0.002300        0.9680             0.1790   
4246          0.000124         -0.000124        1.3860             0.0156   
548           0.000445         -0.000445        3.8552             0.0235   
3953          0.000127         -0.000127        2.9298             0.0124   
2362          0.000987         -0.000987        6.8300             0.0343   

      koi_duration_err2  koi_depth  koi_depth_err2  koi_prad  koi_prad_err1  \
4002            -0.1790     2496.0          -264.0    576.14          30.44   
4246            -0.0156     3992.4           -18.2      6.77           0.51   
548             -0.0235     3450.5           -19.2     38.93          10.98   
3953            -0.0124    37510.0          -100.0     33.24           5.85   
2362            -0.0343      477.1            -3.7      2.26           0.12   

      koi_prad_err2  koi_model_snr  koi_steff_err1  koi_steff_err2  koi_kepmag  
4002         -31.62           12.6             145            -148      15.801  
4246          -0.52          303.8             146            -146      15.653  
548           -7.31          220.3             158            -176      14.039  
3953          -3.59          535.1             122            -134      13.944  
2362          -0.09          134.8              85             -68      10.961

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [9]:
#Scale/Normalize X_train and X_test.
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train Model_5 using SVM



In [10]:
from sklearn.svm import SVC 
model_5 = SVC(kernel='linear')
model_5.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [11]:
model_5.score(X_train_scaled, y_train)

0.8361625023841313

In [12]:
model_5.score(X_test_scaled, y_test)

0.8278032036613272

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [13]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01], 
              'kernel': ["linear", "poly", "rbf"]
             }
grid = GridSearchCV(model_5, param_grid, verbose=3)

In [14]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV] .... C=1, gamma=0.0001, kernel=linear, score=0.851, total=   0.1s
[CV] C=1, gamma=0.0001, kernel=linear ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] .... C=1, gamma=0.0001, kernel=linear, score=0.827, total=   0.1s
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV] .... C=1, gamma=0.0001, kernel=linear, score=0.825, total=   0.1s
[CV] C=1, gamma=0.0001, kernel=linear ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


[CV] .... C=1, gamma=0.0001, kernel=linear, score=0.836, total=   0.1s
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV] .... C=1, gamma=0.0001, kernel=linear, score=0.825, total=   0.1s
[CV] C=1, gamma=0.0001, kernel=poly ..................................
[CV] ...... C=1, gamma=0.0001, kernel=poly, score=0.501, total=   0.5s
[CV] C=1, gamma=0.0001, kernel=poly ..................................
[CV] ...... C=1, gamma=0.0001, kernel=poly, score=0.501, total=   0.5s
[CV] C=1, gamma=0.0001, kernel=poly ..................................
[CV] ...... C=1, gamma=0.0001, kernel=poly, score=0.501, total=   0.5s
[CV] C=1, gamma=0.0001, kernel=poly ..................................
[CV] ...... C=1, gamma=0.0001, kernel=poly, score=0.501, total=   0.5s
[CV] C=1, gamma=0.0001, kernel=poly ..................................
[CV] ...... C=1, gamma=0.0001, kernel=poly, score=0.501, total=   0.5s
[CV] C=1, gamma=0.0001, kernel=rbf ...................................
[CV] .

[CV] ..... C=5, gamma=0.001, kernel=linear, score=0.850, total=   0.1s
[CV] C=5, gamma=0.001, kernel=linear .................................
[CV] ..... C=5, gamma=0.001, kernel=linear, score=0.849, total=   0.1s
[CV] C=5, gamma=0.001, kernel=linear .................................
[CV] ..... C=5, gamma=0.001, kernel=linear, score=0.869, total=   0.1s
[CV] C=5, gamma=0.001, kernel=linear .................................
[CV] ..... C=5, gamma=0.001, kernel=linear, score=0.845, total=   0.1s
[CV] C=5, gamma=0.001, kernel=poly ...................................
[CV] ....... C=5, gamma=0.001, kernel=poly, score=0.501, total=   0.5s
[CV] C=5, gamma=0.001, kernel=poly ...................................
[CV] ....... C=5, gamma=0.001, kernel=poly, score=0.501, total=   0.5s
[CV] C=5, gamma=0.001, kernel=poly ...................................
[CV] ....... C=5, gamma=0.001, kernel=poly, score=0.501, total=   0.5s
[CV] C=5, gamma=0.001, kernel=poly ...................................
[CV] .

[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.749, total=   0.4s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.878, total=   0.1s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.855, total=   0.1s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.853, total=   0.1s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.872, total=   0.2s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.862, total=   0.1s
[CV] C=10, gamma=0.01, kernel=poly ...................................
[CV] ....... C=10, gamma=0.01, kernel=poly, score=0.501, total=   0.5s
[CV] C=10, gamma=0.01, kernel=poly ...................................
[CV] .

[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:   50.9s finished


GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01],
                         'kernel': ['linear', 'poly', 'rbf']},
             verbose=3)

In [15]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
0.8640102533128606


# Save Model_5

In [16]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'model_5.sav'
joblib.dump(model_5, "Model_5.sav")

['Model_5.sav']